In [2]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt

import shapely
from shapely.geometry import Point, Polygon, LineString, GeometryCollection
from shapely.ops import nearest_points 
from shapely import wkt


In [17]:
def get_random_arrangement(n_turbs):

    def get_point():
        """Returns random integer from 50 to 3950 inclusive"""
        return random.uniform(50,3950)

    def is_valid(point):
        """Checks if given point is valid"""
        point = np.array(point)
        point = np.reshape(point,(1,2))
        # getting array of distances to every other point
        dist = np.linalg.norm(turbine_pos - point,axis=1)
        return min(dist) > 400   # 400 is the problem constraint

    turbine_pos = np.full((n_turbs,2),np.inf)
    turb_list = []
    count = 0
    while count < n_turbs:
        point = [get_point(),get_point()] # x,y
        if is_valid(point):
            turbine_pos[count,:] = point
            count += 1
            # turb_list.append(point)
    return turbine_pos

In [30]:
ans = []
for i in range(28):
    a = np.array(pd.read_csv("/home/ananthu/Workspace/Shell AI Hackathon/Trials/opt_swarm_ans{}.csv".format(i)))
    ans.append(a.flatten())
remaining = 64 - len(ans)
for i in range(remaining):
    ans.append(get_random_arrangement(50).flatten())
len(ans)


# ans = np.array(ans)
# # plt.scatter(ans[:,0],ans[:,1])
# # plt.show()

64

In [31]:
b = pd.DataFrame(ans)
b.to_csv("initData.csv", index=False)

In [35]:
a = []
locs = np.array(pd.read_csv("initData.csv"))
for loc in locs:
    a.append(loc.reshape((50,2)))

In [49]:
from sklearn.cluster import KMeans

a_flat = [ind.flatten() for ind in a] 
kmeans = KMeans(n_clusters=5).fit(a_flat)
type(kmeans.cluster_centers_)

numpy.ndarray

In [4]:
A = "EA/"
type(A)

str

In [12]:
B = A + "data"
type(B)
B

'EA/data'